In [ ]:
import pandas as pd
from pyprojroot import here
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = pd.read_csv(here() / 'data/base/data.csv')

In [ ]:
input_data = data.iloc[:,30:43]
throughput_data = data.iloc[:,44:53]
output_data = data.iloc[:,54:59]

In [ ]:
df = data.iloc[:,30:60]

In [ ]:
result = pd.DataFrame({
    col: df[col].value_counts().reindex()
    for col in df.columns
})

In [ ]:
result = result.T
relative = result.div(result.sum(axis=1), axis=0) * 100
relative = relative.round(0)

In [ ]:
relative = relative[['Unimportant', 'Slightly important', 'Moderately important', 'Important', 'Very important']]

In [ ]:
def insert_newline_by_char_limit(sentence, limit):
    """
    Inserts a newline (' \n ') after words so that each line 
    does not exceed the specified number of characters.
    
    Parameters:
        sentence (str): The input sentence.
        limit (int): Maximum number of characters per line.
    
    Returns:
        str: The modified sentence with ' \n ' separating lines.
    """
    words = sentence.split()
    lines = []
    current_line = ""

    for word in words:
        if len(current_line) + len(word) + (1 if current_line else 0) <= limit:
            current_line += (" " if current_line else "") + word
        else:
            lines.append(current_line)
            current_line = word

    if current_line:
        lines.append(current_line)

    return ' \n '.join(lines)

In [ ]:
def insert_newline_in_middle(sentence):
    """
    Inserts a newline character ('\\n') in the middle of the sentence,
    ideally between words, not inside one.
    
    Parameters:
        sentence (str): The input sentence.
    
    Returns:
        str: The sentence with a newline roughly in the middle.
    """
    words = sentence.split()
    if len(words) <= 1:
        return sentence  # not enough words to split meaningfully

    mid_index = len(words) // 2
    first_half = ' '.join(words[:mid_index])
    second_half = ' '.join(words[mid_index:])

    return f"{first_half} \n {second_half}"

In [ ]:
new_index = [insert_newline_in_middle(x) for x in relative.index]
relative.index = new_index

In [ ]:
fig, axs = plt.subplots(
    nrows=3,
    figsize=(2,18),
    gridspec_kw={
        'height_ratios':[14, 9, 7]
    },
    sharex=True
)

inputs = relative[:14]
throughputs = relative[14:23]
outputs = relative[23:]

axs[0].imshow(inputs, aspect='auto', cmap='Greys', vmin=0, vmax=100)
axs[0].set_yticks(np.arange(len(inputs.index)))
axs[0].set_yticklabels(inputs.index)
axs[0].set_title('Input')

axs[1].imshow(throughputs, aspect='auto', cmap='Greys', vmin=0, vmax=100)
axs[1].set_yticks(np.arange(len(throughputs.index)))
axs[1].set_yticklabels(throughputs.index)
axs[1].set_title('Throughput')

axs[2].imshow(outputs, aspect='auto', cmap='Greys', vmin=0, vmax=100)
axs[2].set_yticks(np.arange(len(outputs.index)))
axs[2].set_yticklabels(outputs.index);
axs[2].set_title('Output')


# Annotate each cell with percentage
arr = inputs.values 
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        axs[0].text(j, i, f"{arr[i, j]:.0f}", ha="center", va="center", color="black")

# Annotate each cell with percentage
arr = throughputs.values 
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        axs[1].text(j, i, f"{arr[i, j]:.0f}", ha="center", va="center", color="black")

# Annotate each cell with percentage
arr = outputs.values 
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        axs[2].text(j, i, f"{arr[i, j]:.0f}", ha="center", va="center", color="black")

for ax in axs:
    ax.set_xticklabels(relative.columns, rotation=90)
    ax.set_xticks(list(range(0,5)))

plt.savefig(here() / 'output/plots/causes.png', dpi=300, bbox_inches='tight')